# SQLAlchemy ORM Reflection with Joins

### Here we "reflect" (i.e inspect/read) the design of an existing database.

We read what is out there.  

This of course is awesome for existing databases.

This keeps you from having to manually create classes

In [1]:
# dependencies
import pandas as pd
import numpy as np
# import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData
from sqlalchemy import Table, Column, Integer, String, Float

In [2]:
# connect to database
db_path = "corgies.db"
engine = create_engine(f"sqlite:///{db_path}")
conn = engine.connect()
meta = MetaData()

## What if I just want to know what's out there?

I don't want to update anything, I just want to be able to see the design

In [3]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [4]:
tables = inspector.get_table_names()
print(tables)

['grades', 'pets', 'training']


In [5]:
for table in tables:
    print("\n")
    print('-' * 12)
    print(f"table '{table}' has the following columns:")
    print('-' * 12)
    for column in inspector.get_columns(table):
        print(f"name: {column['name']}   column type: {column['type']}")
        



------------
table 'grades' has the following columns:
------------
name: pet_id   column type: INTEGER
name: task_id   column type: INTEGER
name: grade   column type: TEXT
name: comments   column type: TEXT


------------
table 'pets' has the following columns:
------------
name: id   column type: INTEGER
name: name   column type: TEXT
name: age   column type: INTEGER
name: breed   column type: TEXT
name: age_adopted   column type: TEXT


------------
table 'training' has the following columns:
------------
name: id   column type: INTEGER
name: task   column type: TEXT
name: description   column type: TEXT


## I NEED MORE POWER !!!!

Things have got to change

In [6]:
# let's create the base 
Base = automap_base()

In [7]:
# let's find out what's in this darn database !
Base.prepare(engine, reflect=True)

In [8]:
tables = Base.classes.keys()
tables

['grades', 'pets', 'training']

### Classes = Tables
Remember classes are representations of tables in our database.  

So by getting the classe names, we are getting the table names

# EXPLORE ORM 

## ( with no mamby pamby pandas [except for display readability])

In [9]:
Grades = Base.classes['grades']
Pets = Base.classes['pets']
Training = Base.classes['training']

In [10]:
# remember we need a session for ORM
session = Session(engine)

In [11]:
# idea base from: https://riptutorial.com/sqlalchemy/example/6614/converting-a-query-result-to-dict
def object_as_dict(obj):
    """
    This function takes in a Class instance (row ) from an ORM query and converts it to a dictionary
    """
    obj_count = 1
    try:
        obj_count = len(obj)
    except:
        pass
    if  obj_count == 1:
        base_dict = {c.key: getattr(obj, c.key)
            for c in inspect(obj).mapper.column_attrs}
        return base_dict
    else:
        cur_obj = obj[0]
        base_dict = {c.key: getattr(cur_obj, c.key) for c in inspect(cur_obj).mapper.column_attrs}
        for i in range(1, obj_count):
            cur_obj = obj[i]
            cur_dict = {c.key: getattr(cur_obj, c.key) for c in inspect(cur_obj).mapper.column_attrs}
            base_dict = {**base_dict, **cur_dict} 
        return base_dict    

In [12]:
# from jeff LOL
def query_to_list_of_dicts(cur_query):
    """
    From a query object return a list of dictionaries
    """
    return [object_as_dict(row) for row in cur_query]

In [13]:
pets = query_to_list_of_dicts(session.query(Pets))
# list of dictionaries
pets

[{'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks'},
 {'id': 2,
  'name': 'Judson',
  'age': 10,
  'breed': 'Corgi',
  'age_adopted': '3 years'},
 {'id': 3, 'name': 'Emily', 'age': '?', 'breed': 'Corgi', 'age_adopted': None}]

In [14]:
# easily converted to dataframe
pd.DataFrame(pets)

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years
2,3,Emily,?,Corgi,None


In [15]:
training = query_to_list_of_dicts(session.query(Training))
# for display purposes only
pd.DataFrame(training)

,id,task,description
0,1,sit,sit with bottom on floor
1,2,I feel faint,"lay flat on back legs up like a ""faint"""
2,3,roll over,completely roll over
3,4,speak,bark loud and clear
4,5,lay down,lay with all four legs relaxed


In [16]:
grades = query_to_list_of_dicts(session.query(Grades))
# for display purposes only
pd.DataFrame(grades)

,pet_id,task_id,grade,comments
0,1,1,A,good
1,1,2,A-,"quick but needed ""do a good one"""
2,1,3,B,weak attempt
3,1,4,B,quiet
4,1,5,C,reluctant
5,2,1,B,required butt bump
6,2,2,B-,more of a swoon
7,2,4,A,loud and deep
8,2,5,A,good


# Basic Join

## Engine Inner Join

In [17]:
inner_join_sql = """
select p.*, g.* 
from pets p 
join grades g on p.id = g.pet_id
"""
engine.execute(inner_join_sql).fetchall()

[(1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 1, 'A', 'good'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 2, 'A-', 'quick but needed "do a good one"'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 3, 'B', 'weak attempt'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 4, 'B', 'quiet'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 5, 'C', 'reluctant'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 1, 'B', 'required butt bump'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 2, 'B-', 'more of a swoon'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 4, 'A', 'loud and deep'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 5, 'A', 'good')]

## ORM Inner Join

In [18]:
results = session.query(Pets, Grades).filter(Pets.id == Grades.pet_id)
# for display purposes only
pd.DataFrame(query_to_list_of_dicts(results))

,id,name,age,breed,age_adopted,pet_id,task_id,grade,comments
0,1,Patterson,11,Corgi,8 weeks,1,1,A,good
1,1,Patterson,11,Corgi,8 weeks,1,2,A-,"quick but needed ""do a good one"""
2,1,Patterson,11,Corgi,8 weeks,1,3,B,weak attempt
3,1,Patterson,11,Corgi,8 weeks,1,4,B,quiet
4,1,Patterson,11,Corgi,8 weeks,1,5,C,reluctant
5,2,Judson,10,Corgi,3 years,2,1,B,required butt bump
6,2,Judson,10,Corgi,3 years,2,2,B-,more of a swoon
7,2,Judson,10,Corgi,3 years,2,4,A,loud and deep
8,2,Judson,10,Corgi,3 years,2,5,A,good


# Let's just explore what's going on a step at a time

I want to understand how all the pieces play out

In [19]:
# note how a query is a special object type
# this is essentially an object that auto generates sql a "Query"
session.query(Pets, Grades).filter(Pets.id == Grades.pet_id)

In [20]:
# let's iterate throught this and see what each "row" type is:
# In this case because of the join we are actually returning two classes (pets, grades)
results = session.query(Pets, Grades).filter(Pets.id == Grades.pet_id)
for result in results:
    print(result)
    break

(<sqlalchemy.ext.automap.pets object at 0x000001CD5B32BB08>, <sqlalchemy.ext.automap.grades object at 0x000001CD5B32BC48>)


## Exploring the query itself

In [21]:
# there is a "statement" property
results.statement

<sqlalchemy.sql.selectable.Select at 0x1cd5b326b88; Select object>

In [22]:
# can I see the actual sql being generated?  YES!!!
str(results.statement)

'SELECT pets.id, pets.name, pets.age, pets.breed, pets.age_adopted, grades.pet_id, grades.task_id, grades.grade, grades.comments \nFROM pets, grades \nWHERE pets.id = grades.pet_id'

# Outer Joins

## Engine Version

Note the "None" values in the cells

In [23]:
outer_join_sql = """
select p.*, g.* 
from pets p 
left join grades g on p.id = g.pet_id
"""
engine.execute(outer_join_sql).fetchall()

[(1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 1, 'A', 'good'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 2, 'A-', 'quick but needed "do a good one"'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 3, 'B', 'weak attempt'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 4, 'B', 'quiet'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 5, 'C', 'reluctant'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 1, 'B', 'required butt bump'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 2, 'B-', 'more of a swoon'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 4, 'A', 'loud and deep'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 5, 'A', 'good'),
 (3, 'Emily', '?', 'Corgi', None, None, None, None, None)]

## Let's explore the engine a step at a time

In [24]:
#  Totally new class ResultProxy
engine.execute(outer_join_sql)

In [25]:
# what does a row look like in a ResultProxy?
#
# FOFLOL should have seen that one coming "Row Proxy"
results = engine.execute(outer_join_sql)
for row in results:
    print(row)
    print(type(row))
    break

(1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 1, 'A', 'good')
<class 'sqlalchemy.engine.result.RowProxy'>


In [26]:
# get at that data with dictionary comprehension:
{key: value for key, value in row.items()}

{'id': 1,
 'name': 'Patterson',
 'age': 11,
 'breed': 'Corgi',
 'age_adopted': '8 weeks',
 'pet_id': 1,
 'task_id': 1,
 'grade': 'A',
 'comments': 'good'}

In [27]:
# let's get fancy and embed a dictionary comprehension into a list comprehension
results = engine.execute(outer_join_sql)
# Python comprehensions ROCK!!!
[ {key: value for key, value in row.items()} for row in results]

[{'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks',
  'pet_id': 1,
  'task_id': 1,
  'grade': 'A',
  'comments': 'good'},
 {'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks',
  'pet_id': 1,
  'task_id': 2,
  'grade': 'A-',
  'comments': 'quick but needed "do a good one"'},
 {'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks',
  'pet_id': 1,
  'task_id': 3,
  'grade': 'B',
  'comments': 'weak attempt'},
 {'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks',
  'pet_id': 1,
  'task_id': 4,
  'grade': 'B',
  'comments': 'quiet'},
 {'id': 1,
  'name': 'Patterson',
  'age': 11,
  'breed': 'Corgi',
  'age_adopted': '8 weeks',
  'pet_id': 1,
  'task_id': 5,
  'grade': 'C',
  'comments': 'reluctant'},
 {'id': 2,
  'name': 'Judson',
  'age': 10,
  'breed': 'Corgi',
  'age_adopted': '3 years',
  'pet_id': 2,
  'task_id': 1,
  'grade'

## ORM Outer Join

Note how the "None" type creates challenges that must be addressed.

Currently the query_to_list_of_dicts function breaks upon None types.

In [28]:
results = session.query(Pets, Grades).outerjoin(Grades, Pets.id == Grades.pet_id).all()
for pet, grade in results:
    if grade != None:
        print(f"Pet: {pet.name} received an {grade.grade} on task: {grade.task_id}")
    else:
        print(f"Pet: {pet.name} has taken no training to be graded")


Pet: Patterson received an A on task: 1
Pet: Patterson received an A- on task: 2
Pet: Patterson received an B on task: 3
Pet: Patterson received an B on task: 4
Pet: Patterson received an C on task: 5
Pet: Judson received an B on task: 1
Pet: Judson received an B- on task: 2
Pet: Judson received an A on task: 4
Pet: Judson received an A on task: 5
Pet: Emily has taken no training to be graded


# Views

### They create interesting challenges

They are not a table ORM is all about tables

ORM requieres indexes in general

Results come back as list of tuples

In [29]:
# reflect the view
PetTraining = Table('pet_training', meta, 
                    Column('pet_id', Integer, primary_key=True),
                    Column('task_id', Integer, primary_key=True),
                    autoload_with=engine)


In [30]:
results = session.query(PetTraining)
results = results.filter_by(name='Patterson')
results = results.filter_by(grade='B')
results.all()

[(1, 'Patterson', 3, 'roll over', 'B', 'weak attempt'),
 (1, 'Patterson', 4, 'speak', 'B', 'quiet')]

In [31]:
# get the columns
training_columns = [column.name for column in PetTraining.columns]

#### This looks a whole lot like Result and Row Proxy from engine methods

In [32]:
# exactly what is this query:
session.query(PetTraining)

In [33]:
# do we still have statement?
results = session.query(PetTraining)
results.statement

<sqlalchemy.sql.selectable.Select at 0x1cd5b2137c8; Select object>

In [34]:
# we can see the query as usual
str(results.statement)

'SELECT pet_training.pet_id, pet_training.name, pet_training.task_id, pet_training.task, pet_training.grade, pet_training.comments \nFROM pet_training'

# JOINS ORM AND PANDAS OH MY

Based on what we now know about ORM, Engine and SQLAlchemy objects Panda DataFrames could quickly become our hero.

#### Remember no matter what we do with SQLAlchemy we get a query object back that we can get the sql statement from

###  ENGINE USAGE AND PANDAS

In [42]:
outer_join_sql

'\nselect p.*, g.* \nfrom pets p \nleft join grades g on p.id = g.pet_id\n'

In [35]:
# since engine methods just take sql, it's super simple
pd.read_sql(outer_join_sql, conn)

,id,name,age,breed,age_adopted,pet_id,task_id,grade,comments
0,1,Patterson,11,Corgi,8 weeks,1.0,1.0,A,good
1,1,Patterson,11,Corgi,8 weeks,1.0,2.0,A-,"quick but needed ""do a good one"""
2,1,Patterson,11,Corgi,8 weeks,1.0,3.0,B,weak attempt
3,1,Patterson,11,Corgi,8 weeks,1.0,4.0,B,quiet
4,1,Patterson,11,Corgi,8 weeks,1.0,5.0,C,reluctant
5,2,Judson,10,Corgi,3 years,2.0,1.0,B,required butt bump
6,2,Judson,10,Corgi,3 years,2.0,2.0,B-,more of a swoon
7,2,Judson,10,Corgi,3 years,2.0,4.0,A,loud and deep
8,2,Judson,10,Corgi,3 years,2.0,5.0,A,good
9,3,Emily,?,Corgi,None,NaN,NaN,None,None


### JOIN ORM AND PANDAS

In [36]:
# from our query, get the statement for pandas DataFrames
results = session.query(Pets, Grades).outerjoin(Grades, Pets.id == Grades.pet_id)
pd.read_sql(results.statement, conn)

,id,name,age,breed,age_adopted,pet_id,task_id,grade,comments
0,1,Patterson,11,Corgi,8 weeks,1.0,1.0,A,good
1,1,Patterson,11,Corgi,8 weeks,1.0,2.0,A-,"quick but needed ""do a good one"""
2,1,Patterson,11,Corgi,8 weeks,1.0,3.0,B,weak attempt
3,1,Patterson,11,Corgi,8 weeks,1.0,4.0,B,quiet
4,1,Patterson,11,Corgi,8 weeks,1.0,5.0,C,reluctant
5,2,Judson,10,Corgi,3 years,2.0,1.0,B,required butt bump
6,2,Judson,10,Corgi,3 years,2.0,2.0,B-,more of a swoon
7,2,Judson,10,Corgi,3 years,2.0,4.0,A,loud and deep
8,2,Judson,10,Corgi,3 years,2.0,5.0,A,good
9,3,Emily,?,Corgi,None,NaN,NaN,None,None


### ORM VIEWS AND PANDAS

In [37]:
results = session.query(PetTraining)
pd.read_sql(results.statement, conn)

,pet_id,name,task_id,task,grade,comments
0,1,Patterson,1,sit,A,good
1,1,Patterson,2,I feel faint,A-,"quick but needed ""do a good one"""
2,1,Patterson,3,roll over,B,weak attempt
3,1,Patterson,4,speak,B,quiet
4,1,Patterson,5,lay down,C,reluctant
5,2,Judson,1,sit,B,required butt bump
6,2,Judson,2,I feel faint,B-,more of a swoon
7,2,Judson,4,speak,A,loud and deep
8,2,Judson,5,lay down,A,good


# BUT WHAT ABOUT APIs?

- Web sites don't read DataFrames
- You don't read or write APIs to send or receive DataFrames

In [38]:
# let's get the dataframe from above all in one line:
training_data = pd.read_sql(session.query(PetTraining).statement, conn)
training_data.head()

,pet_id,name,task_id,task,grade,comments
0,1,Patterson,1,sit,A,good
1,1,Patterson,2,I feel faint,A-,"quick but needed ""do a good one"""
2,1,Patterson,3,roll over,B,weak attempt
3,1,Patterson,4,speak,B,quiet
4,1,Patterson,5,lay down,C,reluctant


In [39]:
# DataFrames can be converted to list of dictionaries 
# which we can then jsonify via flask or other jsonify tool

training_data.to_dict(orient="records")

[{'pet_id': 1,
  'name': 'Patterson',
  'task_id': 1,
  'task': 'sit',
  'grade': 'A',
  'comments': 'good'},
 {'pet_id': 1,
  'name': 'Patterson',
  'task_id': 2,
  'task': 'I feel faint',
  'grade': 'A-',
  'comments': 'quick but needed "do a good one"'},
 {'pet_id': 1,
  'name': 'Patterson',
  'task_id': 3,
  'task': 'roll over',
  'grade': 'B',
  'comments': 'weak attempt'},
 {'pet_id': 1,
  'name': 'Patterson',
  'task_id': 4,
  'task': 'speak',
  'grade': 'B',
  'comments': 'quiet'},
 {'pet_id': 1,
  'name': 'Patterson',
  'task_id': 5,
  'task': 'lay down',
  'grade': 'C',
  'comments': 'reluctant'},
 {'pet_id': 2,
  'name': 'Judson',
  'task_id': 1,
  'task': 'sit',
  'grade': 'B',
  'comments': 'required butt bump'},
 {'pet_id': 2,
  'name': 'Judson',
  'task_id': 2,
  'task': 'I feel faint',
  'grade': 'B-',
  'comments': 'more of a swoon'},
 {'pet_id': 2,
  'name': 'Judson',
  'task_id': 4,
  'task': 'speak',
  'grade': 'A',
  'comments': 'loud and deep'},
 {'pet_id': 2,
  '

In [40]:
# but is there a one stop shop?

training_data.to_json(orient="records")

'[{"pet_id":1,"name":"Patterson","task_id":1,"task":"sit","grade":"A","comments":"good"},{"pet_id":1,"name":"Patterson","task_id":2,"task":"I feel faint","grade":"A-","comments":"quick but needed \\"do a good one\\""},{"pet_id":1,"name":"Patterson","task_id":3,"task":"roll over","grade":"B","comments":"weak attempt"},{"pet_id":1,"name":"Patterson","task_id":4,"task":"speak","grade":"B","comments":"quiet"},{"pet_id":1,"name":"Patterson","task_id":5,"task":"lay down","grade":"C","comments":"reluctant"},{"pet_id":2,"name":"Judson","task_id":1,"task":"sit","grade":"B","comments":"required butt bump"},{"pet_id":2,"name":"Judson","task_id":2,"task":"I feel faint","grade":"B-","comments":"more of a swoon"},{"pet_id":2,"name":"Judson","task_id":4,"task":"speak","grade":"A","comments":"loud and deep"},{"pet_id":2,"name":"Judson","task_id":5,"task":"lay down","grade":"A","comments":"good"}]'

In [41]:
# always clean up after yourself
session.close()